In [1]:
%matplotlib widget

In [2]:
import numpy as np
import ipyvolume as ipv
from scipy.spatial import Delaunay
from scipy.spatial.transform import Rotation as rot

# plotting
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import
import matplotlib.pyplot as plt

# interactive plots
import ipywidgets as widgets

# from ipywidgets import VBox, HBox, IntSlider, Checkbox, interactive_output, FloatSlider
from IPython.display import display

import libo
from libo.io import tc3

import weldx.geometry as geo
import weldx.transformations as tf
import weldx.utility as ut
import weldx.visualization as vs

# Setup

In [3]:
system_data_index = 17519
scan_data_scan_idx = 17507
scan_data_tcp_idx = 17518

profile_raster_width = 4
trace_raster_width = 4

# Import and process experimental data

In [7]:
# import machine data
ds_system_data_base = tc3.to_xarray(tc3.read_db(system_data_index))

# import scan data
scan_data_layer_0 = libo.io.tools.merge_scan_tcp(scan=scan_data_scan_idx, tcp=17518)

# extract system movement data
ds_movement_data = ds_system_data_base[["FB_X", "FB_Y", "FB_Z", "FB_Rx", "FB_Ry", "FB_Rz"]]

# extract welding movement data
ds_welding_movement_data = ds_movement_data.where(ds_system_data_base.trigSchweissen == 1).dropna("time")
welding_coordinates_base = ds_welding_movement_data.to_array().data[:3, :]
welding_angles_degree_base = ds_welding_movement_data.to_array().data[3:, :]

# extract scanner movement data
scanner_tcp_layer_0_coordinates_base = scan_data_layer_0.scan_tcp.sel(tcp_variable=["X", "Y", "Z"]).data
scanner_tcp_layer_0_angles_degree_base = scan_data_layer_0.scan_tcp.sel(tcp_variable=["Rx", "Ry", "Rz"]).data

# Define coordinate systems

- `cs_base` is the root coordinate system
- `cs_ref` is defined in `cs_base` -> coordinate system created from 3 robot positions
- `cs_sp` is defined in `cs_ref` -> specimen coordinate system with x pointing into the direction of the trace

All data will be transformed to the specimens coordinate system. Data which is not defined in the specimen coordinate system gets the corresponing coordinate system's suffix attached to the variable name

Important note: Measured data is not perfectly orthogonal. We use the cross product to get a z axis which is perfectly orthogonal (in floating point precision) and recalculate a perfectly orthogonal y-axis. There are certainly better ways to address this problem.

In [12]:
# measured reference coordinate system data
origin = ut.to_float_array([67.311, 1055.137, -132.694])
offset_ox = ut.to_float_array([157.345, 1054.486, -132.949])
offset_oy = ut.to_float_array([69.149, 1366.014, -131.962])

# Calculate orthogonal z axis and construct reference coordinate system from x and z axis
vec_x = offset_ox - origin
vec_y = offset_oy - origin
vec_z = np.cross(vec_x, vec_y)
cs_ref_in_base = tf.LocalCoordinateSystem.construct_from_xz_and_orientation(vec_x=vec_x, vec_z=vec_z, origin=origin)

# create constant coordinate systems
cs_sp_in_ref = tf.LocalCoordinateSystem(basis=tf.rotation_matrix_z(np.pi / 2), origin=[-20, -10, -8])
cs_sp_in_base = cs_sp_in_ref + cs_ref_in_base
cs_scan_in_scanner_tcp = tf.LocalCoordinateSystem(basis=[[1, 0, 0], [0, 1, 0], [0, 0, -1]], origin=[0, 0, 260])

# create time dependent scanner coordinate systems
num_scans_layer_0 = scanner_tcp_layer_0_coordinates_base.shape[0]
scanner_tcp_layer_0_rotation_matrices_base = rot.from_euler(
    angles=scanner_tcp_layer_0_angles_degree_base, seq="xyz", degrees=True
).as_dcm()
cs_array_scan_layer_0_in_sp = []
cs_array_scanner_tcp_layer_0_in_base = []
cs_array_scanner_tcp_layer_0_in_sp = []
for i in range(num_scans_layer_0):
    orientation = scanner_tcp_layer_0_rotation_matrices_base[i]
    coordinates = scanner_tcp_layer_0_coordinates_base[i]
    cs_array_scanner_tcp_layer_0_in_base += [tf.LocalCoordinateSystem(basis=orientation, origin=coordinates)]
    cs_array_scanner_tcp_layer_0_in_sp += [cs_array_scanner_tcp_layer_0_in_base[i] - cs_ref_in_base - cs_sp_in_ref]
    cs_array_scan_layer_0_in_sp += [cs_scan_in_scanner_tcp + cs_array_scanner_tcp_layer_0_in_sp[i]]

# Define data transformations

In [19]:
def coordinates_to_specimen_from_base(coordinates):
    rotation = cs_sp_in_base.orientation.transpose()
    translation = cs_sp_in_base.location[:, np.newaxis]
    return np.matmul(rotation, coordinates - translation)


def coordinates_to_parent_from_child(coordinates, cs_child):
    rotation = cs_child.orientation
    translation = cs_child.location[:, np.newaxis]
    return np.matmul(rotation, coordinates) + translation

# Transform data to specimen coordinate system

In [48]:
welding_coordinates_sp = coordinates_to_specimen_from_base(welding_coordinates_base)

scanned_profiles_layer_0_sp = []
for i in range(num_scans_layer_0):
    scanned_profile_scan = scan_data_layer_0.scan_line.data[i].transpose()
    scanned_profiles_layer_0_sp += [coordinates_to_parent_from_child(scanned_profile_scan, cs_array_scan_layer_0_in_sp[i])]


array([[-20.08206837, -20.08297233, -20.08288492, ...,          nan,
                 nan,          nan],
       [-55.2814472 , -55.03653236, -54.95652248, ...,          nan,
                 nan,          nan],
       [ -5.04773716,  -5.22255057,  -5.20585041, ...,          nan,
                 nan,          nan]])

# Create theoretical geometry

In [58]:
# create points
pt_0 = [40, 8]
pt_1_1 = [np.tan(2 / 18 * np.pi) * 8, 8]
pt_1_2 = [np.tan(3 / 18 * np.pi) * 8, 8]
pt_2 = [0, 0]
pt_3 = [40, 0]

# create shapes
shape_p1_r = geo.Shape().add_line_segments([pt_0, pt_1_1, pt_2])
shape_p2_r = geo.Shape().add_line_segments([pt_0, pt_1_2, pt_2])
shape_p1_l = shape_p1_r.reflect([1, 0])
shape_p2_l = shape_p2_r.reflect([1, 0])

# create profiles
profile_1 = geo.Profile([shape_p1_l, shape_p1_r])
profile_2 = geo.Profile([shape_p2_l, shape_p2_r])

# create variable profile
variable_profile = geo.VariableProfile([profile_1, profile_2], [0, 1], [geo.linear_profile_interpolation_sbs])

# create trace
trace = geo.Trace(geo.LinearHorizontalTraceSegment(350))

# create geometry
geometry = geo.Geometry(variable_profile, trace)

# rasterize geometry
geometry_data_sp = geometry.rasterize(profile_raster_width=profile_raster_width, trace_raster_width=trace_raster_width)

# calculate triangles
triangles = Delaunay(geometry_data_sp.transpose()[:,:2]).simplices

# Plot

In [85]:
ipv.figure()
# we draw the tetrahedron

profile_idx = 1570

scanned_profiles_layer_0_sp = np.array(scanned_profiles_layer_0_sp,float)

mesh = ipv.plot_trisurf(geometry_data_sp[0], geometry_data_sp[1], geometry_data_sp[2], triangles=triangles, color= [0.6,0.6,0.6])
ipv.scatter(welding_coordinates_sp[0], welding_coordinates_sp[1], welding_coordinates_sp[2], size=1, marker="sphere", color=[0.3,0.9,0.7])
s = ipv.scatter(scanned_profiles_layer_0_sp[0::10,0,:], scanned_profiles_layer_0_sp[0::10,1,:], scanned_profiles_layer_0_sp[0::10,2,:], size=1, marker="sphere", color='red')
ipv.animation_control(s)
ipv.xlim(0,350)
ipv.ylim(-30,30)
ipv.zlim(-10,30)
ipv.show()
